In [99]:
#Packages needed
using Optim
using Plots
using StatsBase
using Distributions
using Statistics
using Random
using JuMP, Gurobi, GLPK
using TickTock
using Distributed
using CSV
using DataFrames
using JLD2
using Dates

####### This function just checks the OS and helps automate things instead of needed to copy and paste paths
function checkOS()
    path = pwd()
    if path[1] == '/'
        OS = "Mac"
    else
        OS = "Windows"
    end
    return OS
end
OS = checkOS()
file_path = pwd();
#####Load correct .jl files for functions (accommodates for Mac or Windows)
if OS == "Mac"
    inipath = file_path[1:findlast("Julia_Code",file_path)[end]]*"/"
    include(inipath * "1. Tradeoff Curvers and Value of AM Addition/2. Scripts/OptEvalCombined.jl")
else 
    inipath = file_path[1:findlast("Julia_Code",file_path)[end]] * "\\"
    include(inipath * "1. Tradeoff Curvers and Value of AM Addition\\2. Scripts\\OptEvalCombined.jl")
end

full_run_varied_dem_inputs (generic function with 3 methods)

In [100]:
#adi = collect(range(1,5,length=17))
adi = repeat([3],20)
#cv2 = repeat([1.25],17)
cv2 = collect(range(0.25,5,length=20))

##### For all runs within 0.25 to 5 of both ADI and CV2
#varied = [0.25:0.25:5;]
#adivary = [1:0.25:5;]
#adi = repeat(adivary,20)
#cv2 = repeat([varied[1]],17)
#for i = 2:20
#    cv2 = vcat(cv2,repeat([varied[i]],17))
#end

#adi = [1.2; 3.4; 1.2; 3.4; 4.8]
#cv2 = [0.25; 0.25; 3.45; 3.45; 4.85]

starter = "114_redo_6AMCap"
description = []
for i = 1:length(adi)
    push!(description, starter)
end

In [101]:
###########Test Conditions
avg_dem = 3
#adi = [1.2; 
#      3.4; 
#      1.2; 
#      3.4; 
#      4.8] 
#cv2 = [0.25; 
#       0.25; 
#       3.45;
#       3.45; 
#       4.85] 
#description = ["114_FirstRow_4AMCap";
#                "114_SecondRow_4AMCap";
#                "114_ThirdRow_4AMCap";
#                "114_FourthRow_4AMCap";
#                "114_FifthRow_4AMCap"]
lev = avg_dem .* adi
T = 60  #Time Horizon
G = 1 #Number of products
obs = 180 #number of observations in history
nreal = 500 # number of realizations to generate
LT = [7] 
N=1000 # Number of replications to generate LTD based on
AM_capacity = 6

6

In [102]:
############Supply Chain Design
"""
#Kyle's paper matrix 1-3-2

trans_matrix = [0 25 25 25 0  0;
                0 0  0  0  25 0;
                0 0  0  0  50 50;
                0 0  0  0  0  25;
                0 0  0  0  0  0;
                0 0  0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6]
"""
#Validation SC 1-1-4
trans_matrix = [0 25 0  0  0  0;
                0 0  25 25 25 25;
                0 0  0  0  0  0;
                0 0  0  0  0  0;
                0 0  0  0  0  0;
                0 0  0  0  0  0]

TM = [1]
DC = [2]
SL = [3 4 5 6]
"""
#My network - 1-3-4 distributed?
trans_matrix = [0 25 25 25  0 0  0  0;
                0 0  0  0  25 25 0  0;
                0 0  0  0  25 25 0  0;
                0 0  0  0  0  0  25 25;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6 7 8]
"""
nsup, ncust, nTM, valid_TMrte, Suprte, CtTM, CtS = trans_matrix_math(trans_matrix,TM,DC,SL)

nsup

In [103]:
Int(0b00001)+1

2

In [104]:
####### This cell is to be run if you do not want to run the whole network (good to do when it is a larger network to reduce time)
#114 networks of interest
networks = [1; #No AM
            17; #Cent
            16; #Full Dist
            9; #SL1
            5; #SL2
            3; #SL3
            2; #SL4
]
#networks of interest for 134 network
"""
networks = [1; #no AM
            128; #Full AM
            113; #Centralized
            16; #Distributed
            17; #Iso DC
            4;  #Iso SLs
            20; #Iso Network Side
            13; #only Connected SLs have AM
            97; #Connected DCs have AM
            65; #DC1
            33; #DC2
            17; #DC3
            9; #SL1
            5; #SL2
            3; #SL3
            2] #SL4 
#networks or interest for 136 network
networks = [1; #no AM
            512; #Full AM
            449; #Centralized
            64; #Distributed
            65; #Iso DC
            4;  #Iso SLs
            68; #Iso Network Side
            61; #only Connected SLs have AM
            385; #Connected DCs have AM
            445; #Connected Side
            257; #DC1
            129; #DC2
            65; #DC3
            9; #SL1
            5; #SL2
            3; #SL3
            2] #SL4 

networks = [1; #no AM
            512; #Full AM
            449; #Centralized
            64; #Distributed
            65; #Iso DC
            4;  #Iso SLs
            68; #Iso Network Side
            61; #only Connected SLs have AM
            385; #Connected DCs have AM
            445]; #Connected Side
"""

"networks = [1; #no AM\n            128; #Full AM\n            113; #Centralized\n            16; #Distributed\n            17; #Iso DC\n            4;  #Iso SLs\n            20; #Iso Network Side\n            13; #only Connected SLs have AM\n            97; #Connected DCs have AM\n            65; #DC1\n            33; #DC2\n            17; #DC3\n            9; #SL1\n            5; #SL2\n            3; #SL3\n            2] #SL4 \n#networks or interest for 136 network\nnetworks = [1; #no AM\n            512; #Full AM\n            449; #Centralized\n            64; #Distributed\n            65; #Iso DC\n            4;  #Iso SLs\n            68; #Iso Network Side\n            61; #only Connected SLs have AM\n            385; #Connected DCs have AM\n            445; #Connected Side\n            257; #DC1\n            129; #DC2\n            65; #DC3\n            9; #SL1\n            5; #SL2\n            3; #SL3\n            2] #SL4 \n\nnetworks = [1; #no AM\n            512; #Full AM\n     

In [105]:
full_run_varied_dem_inputs(nsup, ncust, nTM, valid_TMrte, Suprte, CtTM, CtS, avg_dem, adi, cv2, lev, T, G, obs, nreal, LT, N, AM_capacity, description, networks)

idx_Croston = Int64[]
idx_SBA = [1]
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
C:\Users\lmdoudni\Documents\Julia_Code\9. Raw Output Data\2021-11-16_114_redo_6AMCap_3ADI_0_25CV2_3AvgDem.jld2
idx_Croston = Int64[]
idx_SBA = [1]
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
C:\Users\lmdoudni\Documents\Julia_Code\9. Raw Output Data\2021-11-16_114_redo_6AMCap_3ADI_0_5CV2_3AvgDem.jld2
idx_Croston = Int64[]
idx_SBA = [1]
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
C:\Users\lmdoudni\Documents\Julia_Code\9. Raw Output Data\2021-11-16_114_redo_6AMCap_3ADI_0_75CV2_3AvgDem.jld2
idx_Croston = Int64[]
idx_SBA = [1]
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
C:\Users\lmdoudni\Do